In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import from_unixtime, when, col, lit

In [11]:
import json

In [2]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
%reset
import pyspark.sql
session = pyspark.sql.SparkSession.builder \
    .master('spark://10.64.22.198:7077') \
    .appName('Demo') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('py-files','helper.py') \
    .getOrCreate()
    
sqlContext = session

print('SparkSQL sesssion created')

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.
SparkSQL sesssion created


In [4]:
from pyspark.sql.functions import lit
from samples_tthml import *


In [5]:
for s in samples:
    print(BASE,samples[s]['filename'])

../../Files/ttH_Multilep/ mc16_13TeV.aMcAtNloPythia8EvtGen_ttH_r9364_p3832.csv
../../Files/ttH_Multilep/ mc16_13TeV.Sherpa_221_NN30NNLO_ttW_multilegNLO_r9364_p3830.csv


In [28]:
DFList = [] 

for s in samples:
    dsPath = BASE+samples[s]['filename']
    print('Loading {0} sample file'.format(s),'\n',dsPath )   
    tempDF =     spark.read\
                .format('csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .load(dsPath)
    tempDF=tempDF.withColumn("sample", lit(s)) 
    DFList.append(tempDF)

Loading ttH sample file 
 ../../Files/ttH_Multilep/mc16_13TeV.aMcAtNloPythia8EvtGen_ttH_r9364_p3832.csv
Loading ttW sample file 
 ../../Files/ttH_Multilep/mc16_13TeV.Sherpa_221_NN30NNLO_ttW_multilegNLO_r9364_p3830.csv


In [29]:
DFList[0].printSchema()

root
 |-- entry: integer (nullable = true)
 |-- DEtall01: double (nullable = true)
 |-- lep_flavour: integer (nullable = true)
 |-- max_eta: double (nullable = true)
 |-- minDeltaR_LJ_0: double (nullable = true)
 |-- minDeltaR_LJ_1: double (nullable = true)
 |-- Meff: double (nullable = true)
 |-- DRlj00: double (nullable = true)
 |-- min_DRl0b: double (nullable = true)
 |-- min_DRlj_new: double (nullable = true)
 |-- DPhij0MET: double (nullable = true)
 |-- lead_BjetPt: double (nullable = true)
 |-- lead_jet_sumBEff: double (nullable = true)
 |-- sublead_jet_sumBEff: double (nullable = true)
 |-- scale_nom: double (nullable = true)
 |-- pileupEventWeight_090: double (nullable = true)
 |-- MV2c10_70_EventWeight: double (nullable = true)
 |-- JVT_EventWeight: double (nullable = true)
 |-- SherpaNJetWeight: double (nullable = true)
 |-- EventNumber: integer (nullable = true)
 |-- RunYear: integer (nullable = true)
 |-- MET_RefFinal_et: double (nullable = true)
 |-- lep_Pt_0: double (null

In [30]:
total_events = DFList[0].count()
print('There are',total_events,'events')

There are 69902 events


In [32]:
with open("../VarList_30.json") as vardict:
    variablelist = json.load(vardict)[:]
    
variablelist.append("sample")
variablelist

['DEtall01',
 'lep_flavour',
 'max_eta',
 'minDeltaR_LJ_0',
 'minDeltaR_LJ_1',
 'Meff',
 'DRlj00',
 'min_DRl0b',
 'min_DRlj_new',
 'DPhij0MET',
 'lead_BjetPt',
 'lead_jet_sumBEff',
 'sublead_jet_sumBEff',
 'MET_RefFinal_et',
 'lep_Pt_0',
 'lep_Eta_0',
 'lep_Phi_0',
 'lep_Pt_1',
 'lep_Eta_1',
 'lep_Phi_1',
 'total_charge',
 'Mll01',
 'Ptll01',
 'DRll01',
 'nJets_OR_T',
 'nJets_OR_T_MV2c10_70',
 'HT_lep',
 'HT_jets',
 'lead_jetPt',
 'sublead_jetPt',
 'sample']

In [33]:
# Select columns from dataframe
DF = DFList[0].select(variablelist)
DF.printSchema()

root
 |-- DEtall01: double (nullable = true)
 |-- lep_flavour: integer (nullable = true)
 |-- max_eta: double (nullable = true)
 |-- minDeltaR_LJ_0: double (nullable = true)
 |-- minDeltaR_LJ_1: double (nullable = true)
 |-- Meff: double (nullable = true)
 |-- DRlj00: double (nullable = true)
 |-- min_DRl0b: double (nullable = true)
 |-- min_DRlj_new: double (nullable = true)
 |-- DPhij0MET: double (nullable = true)
 |-- lead_BjetPt: double (nullable = true)
 |-- lead_jet_sumBEff: double (nullable = true)
 |-- sublead_jet_sumBEff: double (nullable = true)
 |-- MET_RefFinal_et: double (nullable = true)
 |-- lep_Pt_0: double (nullable = true)
 |-- lep_Eta_0: double (nullable = true)
 |-- lep_Phi_0: double (nullable = true)
 |-- lep_Pt_1: double (nullable = true)
 |-- lep_Eta_1: double (nullable = true)
 |-- lep_Phi_1: double (nullable = true)
 |-- total_charge: integer (nullable = true)
 |-- Mll01: double (nullable = true)
 |-- Ptll01: double (nullable = true)
 |-- DRll01: double (nullab

In [34]:
# Merge all dataset into a single dataframe
for df_ in DFList[1:]:
    DF = DF.union(df_.select(variablelist))

print( 'Partitions: {}'.format(DF.rdd.getNumPartitions()))

Partitions: 2


In [35]:
print('total number of events in the DataFrame  = ', DF.count())
print( 'events in the DataFrame with \"nJets_OR_T > 5\" = ', DF.filter('nJets_OR_T > 5').count())

total number of events in the DataFrame  =  156656
events in the DataFrame with "nJets_OR_T > 5" =  38368


cach dataframe into memory, shared across the Spark cluster nodes

In [36]:
DF = DF.cache()

In [39]:
DF.filter(DF['sample'] == 'ttH')\
  .select('sample','nJets_OR_T','lep_Pt_0','lep_Eta_0','lep_Phi_0','total_charge')\
  .show(5)

+------+----------+---------+-----------+-----------+------------+
|sample|nJets_OR_T| lep_Pt_0|  lep_Eta_0|  lep_Phi_0|total_charge|
+------+----------+---------+-----------+-----------+------------+
|   ttH|         6| 65184.87|  1.3295902|   2.048432|          -2|
|   ttH|         5|40491.316|-0.41755068|-0.65665025|           2|
|   ttH|         4| 80014.03|0.043774348| -1.9746306|          -2|
|   ttH|         5|35063.797|  1.3237612|   3.133612|           2|
|   ttH|         7| 45864.24|  1.9134886| -1.6473318|           2|
+------+----------+---------+-----------+-----------+------------+
only showing top 5 rows

